In [1]:
import numpy as np
import argparse
import sys
import cv2
from math import pow, sqrt

In [2]:
confidence_thresh = 0.85

In [3]:
labels = [line.strip() for line in open('class_labels.txt')]

In [4]:
# Generate random bounding box bounding_box_color for each label

bounding_box_color = np.random.uniform(0, 255, size=(len(labels), 3))

In [5]:
# Load model

print("\nLoading model...\n")
network = cv2.dnn.readNetFromCaffe('SSD_MobileNet_prototxt.txt', 'SSD_MobileNet.caffemodel')

print("\nStreaming video using device...\n")


Loading model...


Streaming video using device...



In [6]:
# Capture video from file or through device
video = 'test_sample.mp4'
if video:
    cap = cv2.VideoCapture(video)
else:
    cap = cv2.VideoCapture(0)

In [7]:
frame_no = 0

while cap.isOpened():

    frame_no = frame_no+1

    # Capture one frame after another
    ret, frame = cap.read()

    if not ret:
        break

    (h, w) = frame.shape[:2]

    # Resize the frame to suite the model requirements. Resize the frame to 300X300 pixels
    blob = cv2.dnn.blobFromImage(cv2.resize(frame, (300, 300)), 0.007843, (300, 300), 127.5)

    network.setInput(blob)
    detections = network.forward()

    # x,y,z in cm
    pos_dict = dict()
    # x,y offset coordinates 
    coordinates = dict()

    # Focal length
    F = 615
    
    # loop over the detections
    for i in range(detections.shape[2]):

        # extract the probability associated with the prediction
        confidence = detections[0, 0, i, 2]

        if confidence > confidence_thresh:

            class_id = int(detections[0, 0, i, 1])

            # compute the (x, y)-coordinates of the bounding box for the object
            box = detections[0, 0, i, 3:7] * np.array([w, h, w, h])
            (startX, startY, endX, endY) = box.astype('int')

            # Filtering only persons detected in the frame. Class Id of 'person' is 15
            if class_id == 15.00:

                # Draw bounding box for the object
                cv2.rectangle(frame, (startX, startY), (endX, endY), bounding_box_color[class_id], 2)

                label = "{}: {:.2f}%".format(labels[class_id], confidence * 100)
                print("{}".format(label))


                coordinates[i] = (startX, startY, endX, endY)

                # Mid point of bounding box
                x_mid = round((startX+endX)/2,4)
                y_mid = round((startY+endY)/2,4)

                height = round(endY-startY,4)

                # Distance from camera based on triangle similarity
                distance = (165 * F)/height
                print("Distance(cm):{dist}\n".format(dist=distance))

                # Mid-point of bounding boxes (in cm) based on triangle similarity technique
                x_mid_cm = (x_mid * distance) / F
                y_mid_cm = (y_mid * distance) / F
                pos_dict[i] = (x_mid_cm,y_mid_cm,distance)

    # Distance between every object detected in a frame
    close_objects = set()
    for i in pos_dict.keys():
        for j in pos_dict.keys():
            if i < j:
                dist = sqrt(pow(pos_dict[i][0]-pos_dict[j][0],2) + pow(pos_dict[i][1]-pos_dict[j][1],2) + pow(pos_dict[i][2]-pos_dict[j][2],2))

                # Check if distance less than 2 metres or 200 centimetres
                if dist < 800:
                    close_objects.add(i)
                    close_objects.add(j)

    for i in pos_dict.keys():
        if i in close_objects:
            COLOR = (0,0,255)
        else:
            COLOR = (0,255,0)
        (startX, startY, endX, endY) = coordinates[i]

        cv2.rectangle(frame, (startX, startY), (endX, endY), COLOR, 2)
        y = startY - 15 if startY - 15 > 15 else startY + 15
        # Convert cms to feet
        cv2.putText(frame, 'Depth: {i} ft'.format(i=round(pos_dict[i][2]/30.48,4)), (startX, y),
                    cv2.FONT_HERSHEY_SIMPLEX, 0.5, COLOR, 2)

    cv2.namedWindow('Frame',cv2.WINDOW_NORMAL)

    # Show frame
    cv2.imshow('Frame', frame)
    cv2.resizeWindow('Frame',800,600)

    key = cv2.waitKey(3) & 0xFF

    # Press `q` to exit
    if key == ord("q"):
        break

# Clean
cap.release()
cv2.destroyAllWindows()

person: 86.41%
Distance(cm):774.6183206106871

person: 87.06%
Distance(cm):774.6183206106871

person: 85.70%
Distance(cm):805.3571428571429

person: 85.10%
Distance(cm):774.6183206106871

person: 85.90%
Distance(cm):780.5769230769231

person: 88.20%
Distance(cm):805.3571428571429

person: 88.01%
Distance(cm):805.3571428571429

person: 86.34%
Distance(cm):805.3571428571429

person: 89.98%
Distance(cm):799.0157480314961

person: 90.86%
Distance(cm):792.7734375

person: 87.01%
Distance(cm):799.0157480314961

person: 89.51%
Distance(cm):792.7734375

person: 87.35%
Distance(cm):792.7734375

person: 87.20%
Distance(cm):774.6183206106871

person: 90.80%
Distance(cm):780.5769230769231

person: 92.58%
Distance(cm):774.6183206106871

person: 87.52%
Distance(cm):786.6279069767442

person: 86.73%
Distance(cm):780.5769230769231

person: 86.26%
Distance(cm):786.6279069767442

person: 88.79%
Distance(cm):799.0157480314961

person: 93.73%
Distance(cm):774.6183206106871

person: 85.16%
Distance(cm):805

KeyboardInterrupt: 